In [1]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
%matplotlib inline

# 一 距离函数
## 1.1 余弦相似度

In [4]:
x1=torch.arange(1,9,dtype=torch.float).reshape(2,4)
x2=torch.arange(10,18,dtype=torch.float).reshape(2,4)
print(x1,'\n',x2)

tensor([[1., 2., 3., 4.],
        [5., 6., 7., 8.]]) 
 tensor([[10., 11., 12., 13.],
        [14., 15., 16., 17.]])


In [5]:
m=nn.CosineSimilarity()
m(x1,x2)

tensor([0.9481, 0.9952])

In [5]:
numerator=torch.dot(x1[0,:],x2[0,:])

In [6]:
denominator=torch.sqrt(torch.sum(x1[0,:]**2))*torch.sqrt(torch.sum(x2[0,:]**2))

In [7]:
numerator/denominator

tensor(0.9481)

## 1.2 空间距离

In [6]:
m=nn.PairwiseDistance()##默认为欧几里得距离，p=2
m(x1,x2)

tensor([18.0000, 18.0000])

In [12]:
type(x1.grad_fn)

NoneType

In [13]:
x1.is_leaf

True

In [14]:
x1.requires_grad

False

# 二 损失函数
## 2.1 L1损失

In [15]:
y=torch.arange(8,dtype=torch.float).reshape(2,4)
t=y+1
t

tensor([[1., 2., 3., 4.],
        [5., 6., 7., 8.]])

In [16]:
loss_fn=nn.L1Loss()##默认使用elementwise_mean
loss=loss_fn(t,y)
loss##注意输出为一个标量

tensor(1.)

In [22]:
loss_fn=nn.L1Loss(reduction='sum')
loss=loss_fn(t,y)
loss##输出为一个标量

tensor(8.)

## 2.2 均方误差MSELoss

In [33]:
loss_fn=nn.MSELoss()
t=y+2
t[1,1]=10
loss_fn(t,y)

tensor(6.6250)

__输出的总是标量，没有batch_size维__

In [34]:
target=torch.empty(3).random_(2)
target

tensor([0., 0., 1.])

## 2.3 二分类交叉熵损失：BCELoss

In [63]:
E=nn.BCELoss()
m=nn.Sigmoid()
y=torch.randn(3,dtype=torch.float,requires_grad=True)
target=torch.empty(3).random_(2)
y

tensor([ 0.8589,  1.0864, -1.0978], requires_grad=True)

In [64]:
target

tensor([0., 0., 1.])

In [65]:
m(y)

tensor([0.7024, 0.7477, 0.2502], grad_fn=<SigmoidBackward>)

In [71]:
loss=E(m(y),target)##对y进行激活之后，再算交叉熵
loss

tensor(1.3250, grad_fn=<BinaryCrossEntropyBackward>)

In [67]:
loss.backward()

In [68]:
y.grad

tensor([ 0.2341,  0.2492, -0.2499])

In [72]:
##手动计算
r=-torch.log(m(y))
r2=-torch.log(1-m(y))
r3=r2[0]+r2[1]+r[2]
r3/3

tensor(1.3250, grad_fn=<DivBackward0>)

### 2.3.1 二分类交叉熵手动计算说明

$E(w)=-\sum_{n}^{N}(t_{n}\log y_{n}+(1-t_{n})\log (1-y_{n})) $

$\bar{E(w)}=\frac{E(w)}{N}$

- 对m(y)和1-m(y)分别取负对数，记为r,r2
- 如果target[n]=1，则取r[n]，否则取r2[n]
- 上述相加除N

## 2.4 负对数似然损失NLLLoss

### 2.4.1.1 一维示例

In [47]:
E=nn.NLLLoss()
m=nn.LogSoftmax(dim=1)
y=torch.randn(3,5,dtype=torch.float,requires_grad=True)##N*C，3个样本，5个分类
target=torch.tensor([1,0,4])
y

tensor([[-1.8861,  2.8642,  2.1634,  1.5570,  1.1928],
        [-0.2726, -0.3854,  0.7054, -1.1143,  0.3424],
        [-0.0046,  1.3266,  1.6912, -0.3976, -0.9664]], requires_grad=True)

In [48]:
m(y)

tensor([[-5.4250, -0.6746, -1.3755, -1.9819, -2.3461],
        [-1.9218, -2.0346, -0.9438, -2.7635, -1.3068],
        [-2.4242, -1.0931, -0.7285, -2.8173, -3.3860]],
       grad_fn=<LogSoftmaxBackward>)

In [49]:
loss=E(m(y),target)

In [50]:
loss

tensor(1.9941, grad_fn=<NllLossBackward>)

In [51]:
loss.backward()

In [52]:
y.grad

tensor([[ 0.0015, -0.1636,  0.0842,  0.0459,  0.0319],
        [-0.2846,  0.0436,  0.1297,  0.0210,  0.0902],
        [ 0.0295,  0.1117,  0.1609,  0.0199, -0.3221]])

In [59]:
##手动计算损失
r=m(y)[0,1]+m(y)[1,0]+m(y)[2,4]
-r/3

tensor(1.9941, grad_fn=<DivBackward0>)

### 2.4.1.2 多分类交叉熵手动计算说明
$E(w)=-\sum_{n}^{N}t_{n}\log y_{n}$

$\bar{E(w)}=\frac{E(w)}{N}$

假设 target=torch.tensor([1,0,4])，因随机数每次生成可能不同：

- 第一个样本属于第1类，即$t_{1}=1而t_{k\ne1}=0$，因此只取下标为[0,1]的元素即可
- 同理，第二、三个样本，只取[1,0]、[2,4]即可
- 三者相加取负数除3即可

## 2.4.2 二维示例

In [73]:
N,C=5,4 ##5个样本，4个分类
E=nn.NLLLoss()
X=torch.randn(N,16,10,10)##4个样本，16个通道，H*W=10x10
target=torch.empty(N,8,8).random_(0,C)
target

tensor([[[2., 3., 0., 0., 3., 3., 3., 3.],
         [3., 2., 1., 1., 2., 3., 2., 2.],
         [3., 2., 3., 2., 3., 2., 1., 0.],
         [2., 2., 3., 0., 2., 3., 2., 0.],
         [3., 0., 3., 1., 0., 3., 3., 0.],
         [1., 3., 3., 2., 3., 0., 2., 2.],
         [3., 3., 2., 0., 1., 1., 1., 2.],
         [1., 2., 2., 3., 1., 1., 2., 2.]],

        [[2., 2., 1., 2., 0., 3., 1., 2.],
         [2., 2., 3., 1., 3., 1., 3., 3.],
         [2., 1., 3., 1., 0., 3., 3., 1.],
         [2., 3., 2., 1., 1., 0., 3., 1.],
         [0., 3., 3., 1., 2., 1., 3., 2.],
         [2., 2., 0., 3., 1., 1., 1., 3.],
         [0., 1., 2., 1., 1., 2., 1., 1.],
         [3., 1., 2., 2., 0., 3., 0., 0.]],

        [[0., 2., 0., 0., 2., 0., 1., 2.],
         [2., 2., 0., 1., 3., 3., 2., 0.],
         [0., 1., 2., 0., 3., 3., 1., 1.],
         [2., 2., 2., 0., 3., 0., 1., 2.],
         [0., 2., 3., 2., 2., 1., 2., 3.],
         [3., 0., 2., 1., 0., 0., 1., 0.],
         [1., 1., 3., 2., 2., 3., 0., 0.],
       

In [75]:
##注意卷积核的形状
m=nn.Conv2d(16,C,3)#输入16个通道，输出C=4个通道，核尺寸为3，则输出后尺寸:[(10-3)/1]+1=8
y=m(X)
y.shape

torch.Size([5, 4, 8, 8])

In [79]:
target=target.to(torch.long)
loss=E(y,target)
loss

tensor(0.0063, grad_fn=<NllLoss2DBackward>)

## 2.5 交叉熵损失CrossEntropyLoss

这个损失函数相当于 LogSoftmax+NLLLoss 的混合，即对于输入数据，不必再进行LogSoftmax的变换，直接计算交叉熵损失即可。

In [80]:
E=nn.CrossEntropyLoss()
X=torch.randn(3,5)
target=torch.empty(3,dtype=torch.long).random_(5)
target

tensor([3, 2, 2])

In [81]:
loss=E(X,target)
loss

tensor(1.7148)

In [82]:
X=torch.tensor([[-1.8861,  2.8642,  2.1634,  1.5570,  1.1928],
        [-0.2726, -0.3854,  0.7054, -1.1143,  0.3424],
        [-0.0046,  1.3266,  1.6912, -0.3976, -0.9664]])

target=torch.tensor([1,0,4])

E(X,target)

tensor(1.9942)

__在使用相同数据X的情况下，这个结果与2.4.1.1的输出结果一致__

## 2.6 KL散度损失

In [84]:
import torch.nn.functional as F
E=nn.KLDivLoss()
log_prob1=F.log_softmax(torch.randn(5,10),dim=1)
prob2=F.softmax(torch.randn(5,10),dim=1)
E(log_prob1,prob2)

tensor(0.0686)

## 2.7 带sigmoid激活的二分类交叉熵损失 BCEWithLogitsLoss

与BCELoss相比，自动添加了sigmoid激活

In [89]:
E=nn.BCEWithLogitsLoss()
y=torch.tensor([ 0.8589,  1.0864, -1.0978])
target=torch.tensor([0., 0., 1.])
E(y,target)##注意变换的变量在前，标签在后，顺序不能乱

tensor(1.3250)

In [88]:
E2=nn.BCELoss()
m=nn.Sigmoid()
E2(m(y),target)

tensor(1.3250)

## 2.8 其他损失
smoothl1loss等，详见：https://pytorch.org/docs/stable/nn.html#smoothl1loss